In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pip install dash

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd


In [ ]:
import plotly.graph_objects as go

In [ ]:
import warnings
warnings.filterwarnings("ignore")   

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving covid_19_india.csv to covid_19_india.csv
User uploaded file "covid_19_india.csv" with length 884834 bytes


In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['covid_19_india.csv']))

In [ ]:
df.drop(['Sno','Time', 'ConfirmedIndianNational','ConfirmedForeignNational'], axis=1, inplace=True)

In [ ]:
#Rename columns for easier calling
df.columns = ['date','state','discharged','deaths','confirmed']

In [ ]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')

In [ ]:
df['YearMonth'] = df['date'].apply(lambda x:x.strftime('%Y-%m'))

In [ ]:
df.drop(['date'], axis=1, inplace=True)

In [ ]:
df.head()

,state,discharged,deaths,confirmed,YearMonth
0,Kerala,0,0,1,2020-01
1,Kerala,0,0,1,2020-01
2,Kerala,0,0,2,2020-02
3,Kerala,0,0,3,2020-02
4,Kerala,0,0,3,2020-02


# **Date and State Based Analysis**

In [ ]:
df1 = df.groupby(['YearMonth',"state"])['confirmed','deaths','discharged',].sum().reset_index()

In [ ]:
df1.head()

,YearMonth,state,confirmed,deaths,discharged
0,2020-01,Kerala,2,0,0
1,2020-02,Kerala,86,0,0
2,2020-03,Andaman and Nicobar Islands,36,0,0
3,2020-03,Andhra Pradesh,163,0,7
4,2020-03,Bihar,73,10,0


In [ ]:
fig = px.line(df1, x="YearMonth", y="deaths", color="state")
fig.show()

In [ ]:
fig1 = px.bar(df1, x="YearMonth", y="confirmed", color="discharged")
fig1.show()

In [ ]:
#comparing two states covid rate
country1 = 'Maharashtra'
country1_data = df1.loc[df1['state'] == country1]
country1_cases = country1_data[['YearMonth','confirmed']]

country2 = 'Tamil Nadu'
country2_data = df1.loc[df1['state'] == country2]
country2_cases = country2_data[['YearMonth','confirmed']]

datewise_cases = country1_cases.merge(country2_cases, how = 'inner', on = 'YearMonth') 
print(datewise_cases.tail())

   YearMonth  confirmed_x  confirmed_y
11   2021-02     57992941     23654321
12   2021-03     73873249     26753641
13   2021-04    109094556     29399621
14   2021-05    163559278     49154426
15   2021-06     46374074     17446770


In [ ]:
fig2=go.Figure()
fig2.add_trace(go.Scatter(x=datewise_cases['YearMonth'].values, y=datewise_cases['confirmed_x'].values,
                         mode='lines',
                         name='MH'))
fig2.add_trace(go.Scatter(x=datewise_cases['YearMonth'].values, y=datewise_cases['confirmed_y'].values,
                         mode='lines',
                         name='TN'))

fig2.update_layout(title='Rate of covid Cases')
fig2.show()

# **Analysis based on Date**

In [ ]:
df2 = df.groupby('YearMonth')['confirmed','deaths','discharged',].sum().reset_index()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [ ]:
df2.head()

,YearMonth,confirmed,deaths,discharged
0,2020-01,2,0,0
1,2020-02,86,0,0
2,2020-03,9687,202,808
3,2020-04,422442,13270,75443
4,2020-05,2938234,89834,1133341


In [ ]:
df2['discharge_rate'] = df2['discharged']/df2['confirmed']      # create instances for 'death_rate and discharge_rate'
df2['death_rate'] = df2['deaths']/df2['confirmed']

In [ ]:
df2.replace(np.inf,0,inplace=True)  #infinite values are replaced by -0

In [ ]:
fig2_1 = px.line(df2, x="YearMonth", y="confirmed",)

fig2_1.show()

In [ ]:
fig2_2 = px.line(df2, x="YearMonth", y="discharge_rate",)
fig2_2.update_layout(
    title={
        'text': "Discharge rate",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig2_2.show()

# **Analysis Based on State**

In [ ]:
df3 = df.groupby('state')['confirmed','deaths','discharged',].sum().reset_index()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [ ]:
df3.head()

,state,confirmed,deaths,discharged
0,Andaman and Nicobar Islands,1461311,18944,1381883
1,Andhra Pradesh,270440364,2117572,251842054
2,Arunachal Pradesh,4619157,14682,4257945
3,Assam,66322683,335961,61027980
4,Bihar,87441641,502330,81029353


In [ ]:
#Over all Confirmed cases

fig3 = go.Figure(go.Bar(
            x=df3['confirmed'],
            y=df3['state'],
            orientation='h'))
fig3.update_layout(barmode='stack', yaxis={'categoryorder':'total ascending'})
fig3.show()

In [ ]:
#top 10 States with High Discharge
top_10 = df3.sort_values(by=['confirmed'], ascending=False).head(10)

fig = go.Figure()
fig.add_trace(go.Bar(
    y=top_10['state'],
    x=top_10['confirmed'],
    name='confirmed',
    orientation='h',
    marker=dict(
        color='rgba(246, 78, 139, 0.6)',
        line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
    )
))
fig.add_trace(go.Bar(
    y= top_10['state'],
    x=top_10['discharged'],
    name='Discharged',
    orientation='h',
    marker=dict(
        color='rgba(58, 71, 80, 0.6)',
        line=dict(color='rgba(58, 71, 80, 1.0)', width=3)
    )
))

fig.update_layout(barmode='stack', yaxis={'categoryorder':'total ascending'})
fig.show()